In [1]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import time
import datetime
from selenium import webdriver
from random import randint
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
import selenium.webdriver.support.expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import NoSuchElementException
# Headless Firefox Option
#from selenium.webdriver.firefox.options import Options
#options = Options()
#options.headless = True # This makes it headless

#driver = webdriver.Firefox(options=options)
driver = webdriver.Firefox()
sproutsurl='https://shop.sprouts.com/'
driver.get(sproutsurl)
time.sleep(5)
# Departments: Produce, Deli, Bakery, Bulk, Dairy, Meat&Seafood, Grocery Aisles, Frozen, Vitamins/BodyCare, Beer
departmentclass=['nav-main-shop-category-1','nav-main-shop-category-41','nav-main-shop-category-54','nav-main-shop-category-103',
                 'nav-main-shop-category-124','nav-main-shop-category-67','nav-main-shop-category-132','nav-main-shop-category-218',
                 'nav-main-shop-category-230','nav-main-shop-category-269']

Enter the zip code

In [2]:
search=WebDriverWait(driver,30).until(EC.visibility_of_element_located((By.ID,'shopping-selector-search-cities')))
driver.find_element_by_id('shopping-selector-search-cities')
driver.find_element_by_id('shopping-selector-search-cities').clear()
search.send_keys('90068')
search.send_keys(Keys.RETURN)

Click on the first store on the list

In [3]:
search=WebDriverWait(driver,30).until(EC.visibility_of_element_located((By.CLASS_NAME,'name')))
storenam=driver.find_element_by_class_name('name')
storeaddres=driver.find_element_by_class_name('address')
storename=storenam.text
storeadd=storeaddres.text
#Get all the store buttons to click
buttons=driver.find_elements_by_class_name('store-selector-button.instore')
buttons[0].click()
storestring=storeadd.splitlines()
storeaddress=storestring[0]
phone=storestring[1]
time.sleep(7)

Optional: Click on Produce section

In [36]:
#produce='nav-main-shop-category-1'
#WebDriverWait(driver,30).until(EC.visibility_of_element_located((By.ID,{}))).format(produce)
#section=driver.find_element_by_id(produce)
#section.click()

Scrape the items

In [8]:
timeStamp=[]
itemurl=[]
title=[]
imageurllist=[]
menuname=[]
pricelist=[]
units=[]
unique_attr=[]
sitetypeid=[]
category=[]
subcatname=[]
instore=[]
groceryattr=[]
store_appear=[] #Store ID - store name, address, phone number
sproutsbrand=[]
organicitem=[]
nongmo=[]
begindex=0
endindex=0

for dept in departmentclass:
    WebDriverWait(driver,30).until(EC.visibility_of_element_located((By.ID,dept)))
    department=driver.find_element_by_id(dept)
    department.click()
    time.sleep(5)
    
    # Find the children under the department
    chinga=[]
    children=driver.find_elements_by_class_name('child')
    numchild=len(children)
    for childindex in range(numchild):
        chinga.append(children[childindex].text)

    childindex=0

    while childindex<numchild:
        department.click()
        time.sleep(3)
        children=driver.find_elements_by_class_name('child')
        children[childindex].click()
        time.sleep(4)

        indx=0
        try:
            lastpage=driver.find_element_by_class_name('pagination-last.pager-item')
            pagenum=int(lastpage.text)
        except NoSuchElementException:
            numofpages=driver.find_elements_by_class_name('pagination-page.pager-item')
            pagenum=len(numofpages)
        while indx < pagenum: # Advance the page
            if indx>0:
                advbutton=driver.find_element_by_class_name('button.small.round.icon-after-chevron-right').click()
            else:
                pass
            time.sleep(5)

            page=driver.page_source
            soup = BeautifulSoup(page, 'html.parser')

            # Get product titles & imageurls
            for entry in soup.find_all(name='a', attrs={'class':'cell-image ng-isolate-scope show'}):
                title.append(entry['title'])
                urlindexnum=entry['style'].find('https')
                imageurl=entry['style'][urlindexnum:-1]
                imageurllist.append(imageurl)
                sitetypeid.append('5')
                endindex+=1

            # Get the item's URL by opening each pop-up window
            for item in title[begindex:endindex]:
                try:
                    # Open the item's pop up
                    finditemurl = driver.find_element_by_xpath("//*[@title='{}']".format(item))
                    finditemurl.click()
                    # Get the item's url
                    WebDriverWait(driver,30).until(EC.visibility_of_element_located((By.XPATH,"/html/body/div[1]/div/div/div[2]/div/div[2]/div/div/ul/li[5]/span[2]")))
                    weburl=driver.find_elements_by_xpath("//a[@href]")[5]
                    itemurl.append(weburl.get_attribute("href"))
                    # Get the breadcrumbs
                    page2=driver.page_source
                    soup2 = BeautifulSoup(page2, 'html.parser')
                    depart=soup2.find_all(name='a', attrs={'class':'ng-binding ng-scope'})
                    tag0=depart[0].text
                    child1=soup2.find(name='span', attrs={'class':'ng-binding ng-scope'})
                    tag1=child1.text.replace("\n", "")
                    tag2=depart[1].text
                    if tag2==item:
                        tag2=''
                    else:
                        pass
                    tags=[tag0,tag1,tag2]
                    subcatname.append(tag1)
                    category.append(tag0)
                    unique_attr.append(tags)
                    # Get Sprouts Brand, Organic, Non-GMO, Gluten-Free
                    grocery=[]
                    page2=driver.page_source
                    soup2=BeautifulSoup(page2, 'html.parser')
                    gmo=soup2.find(name='li', attrs={'class':'tag icon icon-non-gmo'})
                    organic=soup2.find(name='li', attrs={'class':'tag icon icon-organic'})
                    brand=soup2.find(name='li', attrs={'class':'tag icon icon-sprouts-brand'})
                    glutenfree=soup2.find(name='li', attrs={'class':'tag icon icon-gluten_free'})
                    try:
                        gmo.text
                        grocery.append(gmo.text.replace("\n", ""))
                    except AttributeError:
                        pass
                    try:
                        organic.text
                        grocery.append(organic.text.replace("\n", ""))
                    except AttributeError:
                        pass
                    try:
                        brand.text
                        grocery.append(brand.text.replace("\n", ""))
                        sproutsbrand.append(brand.text.replace("\n", ""))
                    except AttributeError:
                        sproutsbrand.append('')
                        pass
                    try:
                        glutenfree.text
                        grocery.append(glutenfree.text.replace("\n", ""))
                    except AttributeError:
                        pass
                    groceryattr.append(grocery)
                    # Close the pop-up
                    xbutton=driver.find_element_by_class_name('close').click()
                except NoSuchElementException:
                    itemurl.append('None')

            # Get the prices
            for entry in soup.find_all(name='div', attrs={'class':'cell-prices product-prices'}):
                boolsale = entry.find(name='div', attrs={'aria-hidden':'false'})
                prices=[]
                for price in entry.find_all(name='span', attrs={'class':'amount'}):
                    prices.append(price.text)
                if boolsale is None:  # If item is on sale, pick the new price & units
                    del prices[0]
                    unit=entry.find(name='span', attrs={'class':'unit-price'})
                    if unit is None:
                        unit=entry.find(name='span', attrs={'class':'unit'})
                else:
                    unit=entry.find(name='span', attrs={'class':'unit'})
                    if unit is None:
                        unit=price
                        #unit=entry.find(name='span', attrs={'class':'unit-price'})
                    pass
                timeStamp.append(datetime.datetime.now().strftime("%Y-%m-%d %H:%M"))
                instore.append('1')
                menuname.append('Grocery Store')
                store_appear.append('') #place holder for store ID
                pricelist.append(prices[0])
                units.append(unit.text)
            begindex=endindex
            indx+=1
        print('Finished ' + str(chinga[childindex]))
        childindex+=1

Yellow Bananas
Hass Avocados, 1 EA
Jumbo Hass Avocados, 1 EA
Organic Bananas
Limes, 1 EA
Red Seedless Grapes
Fresh Pineapple, 1 EA
Cantaloupe Melons, 1 EA
Green Seedless Grapes
Large Navel Oranges
Red Mangos, 1 EA


KeyboardInterrupt: 

In [27]:
timestamp=pd.Series(timeStamp)
instores=pd.Series(instore)
menunamelist=pd.Series(menuname)
itemurls=pd.Series(itemurl)
categorylist=pd.Series(category)
subcatlist=pd.Series(subcatname)
unique_attrs=pd.Series(unique_attr)
imageurls=pd.Series(imageurllist)
titles=pd.Series(title)
itemprice=pd.Series(pricelist)
unitslist=pd.Series(units)
sitetypelist=pd.Series(sitetypeid)
brandlist=pd.Series(sproutsbrand)
grocery_attrs=pd.Series(groceryattr)
store_appearance=pd.Series(store_appear)

In [30]:
table=pd.concat([instores,titles,subcatlist,categorylist,menunamelist,itemurls,sitetypelist,timestamp,timestamp,unique_attrs,imageurls,brandlist,grocery_attrs,store_appearance,itemprice,unitslist], axis=1)
table.columns=['in_store','name','subcategory_name','category_name','menu_name','url','site_type_id','created_at','updated_at','unique_attrs','orig_thumbnail_url','brand','grocery_attrs','store_appearance','price','price_unit']
table.dropna(axis=0, subset=['subcategory_name'], inplace=True)
table


,in_store,name,subcategory_name,category_name,menu_name,url,site_type_id,created_at,updated_at,unique_attrs,orig_thumbnail_url,brand,grocery_attrs,store_appearance,price,price_unit
30,1,Organic Red Seedless Grapes,Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/8818/organic-...,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Grapes]",https://assets-prd-spr.unataops.com/web/produc...,,[Organic],,$3.99,/ lb
31,1,Bosc Pears,Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/43975/bosc-pears,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Pears]",https://assets-prd-spr.unataops.com/web/produc...,,[],,$1.69,/ lb
32,1,Red Delicious Apples,Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/7054/red-deli...,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Apples]",https://assets-prd-spr.unataops.com/web/produc...,,[],,$0.99,/ lb
33,1,Pink Lady Apples,Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/43550/pink-la...,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Apples]",https://assets-prd-spr.unataops.com/web/produc...,,[],,$1.99,/ lb
34,1,Organic Honeycrisp Apples,Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/8794/organic-...,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Apples]",https://assets-prd-spr.unataops.com/web/produc...,,[Organic],,$2.99,/ lb
35,1,Holiday Seedless Grapes,Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/7195/holiday-...,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Grapes]",https://assets-prd-spr.unataops.com/web/produc...,,[],,$2.99,/ lb
36,1,"Organic Mini Watermelon, 1 EA",Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/8799/organic-...,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Melons]",https://assets-prd-spr.unataops.com/web/produc...,,[Organic],,$4.99,/ ea
37,1,Mexican Papaya,Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/44277/mexican...,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Tropical Fruit]",https://assets-prd-spr.unataops.com/web/produc...,,[],,$0.98,/ lb
38,1,Ambrosia Apples,Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/7021/ambrosia...,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Apples]",https://assets-prd-spr.unataops.com/web/produc...,,[],,$1.99,/ lb
39,1,Organic Fuji Apples,Fruits,Produce,Grocery Store,https://shop.sprouts.com/product/44293/organic...,5,2019-03-21 11:51,2019-03-21 11:51,"[Produce, Fruits, Apples]",https://assets-prd-spr.unataops.com/web/produc...,,[Organic],,$1.79,/ lb


In [38]:
table.to_pickle("./SproutsLaBrea.pkl")

In [1]:
import pandas as pd
table=pd.read_pickle("./SproutsLaBrea.pkl")

In [39]:
#table.to_csv('SproutsLABrea.csv', index=False)

In [2]:
from sqlalchemy import create_engine
import pyodbc
engine=create_engine('postgresql://deployer:p0877e546c0539179b1fb234649e2aba6691a28b53815b7710a93d65cc21c2059@haystack.cety7wul5z9k.us-west-2.rds.amazonaws.com:5432/haystack3_production')
table.to_sql('sprouts', con=engine, if_exists = "replace")

/anaconda3/lib/python3.7/site-packages/psycopg2/__init__.py:144: UserWarning: The psycopg2 wheel package will be renamed from release 2.8; in order to keep installing from binary please use "pip install psycopg2-binary" instead. For details see: <http://initd.org/psycopg/docs/install.html#binary-install-from-pypi>.
  """)
